In [1]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'one_to_one_dataset.txt'

Using TensorFlow backend.


In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split(' ')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = ' ' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
import json

with open('important_data.txt') as json_file:  
    data = json.load(json_file)
    input_token_index=data['input_index']
    target_token_index=data['target_index']
    print(input_token_index)
    for p in data['input_output']: 
        num_encoder_tokens = p['num_encoder_tokens']
        num_decoder_tokens = p['num_decoder_tokens']
        max_encoder_seq_length = p['max_encoder_seq_length']
        max_decoder_seq_length = p['max_decoder_seq_length']

{' ': 0, '!': 1, '"': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '<': 11, '=': 12, '>': 13, '?': 14, '[': 15, ']': 16, '{': 17, '|': 18, '}': 19, 'ঁ': 20, 'ং': 21, 'ঃ': 22, 'অ': 23, 'আ': 24, 'ই': 25, 'ঈ': 26, 'উ': 27, 'ঊ': 28, 'ঋ': 29, 'এ': 30, 'ঐ': 31, 'ও': 32, 'ঔ': 33, 'ক': 34, 'খ': 35, 'গ': 36, 'ঘ': 37, 'ঙ': 38, 'চ': 39, 'ছ': 40, 'জ': 41, 'ঝ': 42, 'ঞ': 43, 'ট': 44, 'ঠ': 45, 'ড': 46, 'ঢ': 47, 'ণ': 48, 'ত': 49, 'থ': 50, 'দ': 51, 'ধ': 52, 'ন': 53, 'প': 54, 'ফ': 55, 'ব': 56, 'ভ': 57, 'ম': 58, 'য': 59, 'র': 60, 'ল': 61, 'শ': 62, 'ষ': 63, 'স': 64, 'হ': 65, 'া': 66, 'ি': 67, 'ী': 68, 'ু': 69, 'ূ': 70, 'ৃ': 71, 'ে': 72, 'ৈ': 73, 'ো': 74, 'ৌ': 75, '্': 76, 'ৎ': 77, 'ড়': 78, 'ঢ়': 79, 'য়': 80, '০': 81, '১': 82, '২': 83, '৩': 84, '৪': 85, '৫': 86, '৬': 87, '৭': 88, '৮': 89, '৯': 90, '\u200c': 91, '\u200d': 92, '\u200f': 93, '—': 94, '’': 95}


In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
#num_encoder_tokens = len(input_characters)
#num_decoder_tokens = len(target_characters)
#max_encoder_seq_length = max([len(txt) for txt in input_texts])
#max_decoder_seq_length = max([len(txt) for txt in target_texts])

#print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#input_token_index = dict(
#    [(char, i) for i, char in enumerate(input_characters)])
#target_token_index = dict(
  #  [(char, i) for i, char in enumerate(target_characters)])

print(target_token_index)

#encoder_input_data = np.zeros(
   # (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  #  dtype='float32')

#for i, input_text in enumerate(input_texts):
 #   for t, char in enumerate(input_text):
  #      encoder_input_data[i, t, input_token_index[char]] = 1.

Number of unique input tokens: 96
Number of unique output tokens: 98
Max sequence length for inputs: 22
Max sequence length for outputs: 21
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, ':': 11, ';': 12, '<': 13, '=': 14, '>': 15, '?': 16, '[': 17, ']': 18, '{': 19, '|': 20, '}': 21, 'ঁ': 22, 'ং': 23, 'ঃ': 24, 'অ': 25, 'আ': 26, 'ই': 27, 'ঈ': 28, 'উ': 29, 'ঊ': 30, 'ঋ': 31, 'এ': 32, 'ঐ': 33, 'ও': 34, 'ঔ': 35, 'ক': 36, 'খ': 37, 'গ': 38, 'ঘ': 39, 'ঙ': 40, 'চ': 41, 'ছ': 42, 'জ': 43, 'ঝ': 44, 'ঞ': 45, 'ট': 46, 'ঠ': 47, 'ড': 48, 'ঢ': 49, 'ণ': 50, 'ত': 51, 'থ': 52, 'দ': 53, 'ধ': 54, 'ন': 55, 'প': 56, 'ফ': 57, 'ব': 58, 'ভ': 59, 'ম': 60, 'য': 61, 'র': 62, 'ল': 63, 'শ': 64, 'ষ': 65, 'স': 66, 'হ': 67, 'া': 68, 'ি': 69, 'ী': 70, 'ু': 71, 'ূ': 72, 'ৃ': 73, 'ে': 74, 'ৈ': 75, 'ো': 76, 'ৌ': 77, '্': 78, 'ৎ': 79, 'ড়': 80, 'ঢ়': 81, 'য়': 82, '০': 83, '১': 84, '২': 85, '৩': 86, '৪': 87, '৫': 88, '৬': 89, '৭': 90, '৮': 91, '৯': 92, '\u200c': 93, '\u200d': 94, '\

In [7]:
# Restore the model and construct the encoder and decoder.
model = load_model('s2s.h5')

encoder_inputs = model.input[0]   # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output   # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]   # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_5')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


# Decodes an input sequence.  Future work should support beam search.
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


def get_input_data(test_txt):
    #txt = input()
    txt=test_txt
    
    encoder_input_data = np.zeros(
    (len(txt), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
    
    txt = txt.replace(' ', '')
    
    for t, char in enumerate(txt):
        encoder_input_data[0, t, input_token_index[char]] = 1.
        
    return encoder_input_data

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [32]:
word_list=[]
data_path='dataset_corpus_input.txt'
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines:
    word_list=line.split()
    l=len(word_list)
    if(l==2 and word_list[0][0]=='\u200c' ):
        word=word_list[0]
    else:
        word=word_list[1]
    preword=line
    
    for char in word:
        if char not in input_characters:
           # print("nai nai re... "+char)
            word=word.replace(char,"\u200d")
    
    if(len(word)>22):
        continue
    
    input_seq = get_input_data(word)
    decoded_sentence = decode_sequence(input_seq)
    #print('Input sentence:',   word)
    #print('Decoded sentence:', decoded_sentence)
    file = open('many_to_one.txt','a',encoding="utf-8")
    file.write(preword+"\t"+decoded_sentence)

IndexError: list index out of range

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(epochs)
# Run training
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('test_model.h5')

In [15]:
# Next: inference mode (sampling).
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
from keras.models import model_from_json

data_path = 'fra-eng/fra.txt'
print("kire")

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("test_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
#model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#model = load_model('s2s.h5')

input_characters=66 #here Number of unique input tokens(characters): 66
num_encoder_tokens = 66
num_decoder_tokens=67
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[' ']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence




def get_input_data(test_txt):
    #txt = input()
    txt=test_txt
    
    encoder_input_data = np.zeros(
    (len(txt), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
    
    txt = txt.replace(' ', '')
    
    for t, char in enumerate(txt):
        encoder_input_data[0, t, input_token_index[char]] = 1.
        
    return encoder_input_data
    
    
    
    
# Take one sequence (part of the training set)
# for trying out decoding.
#input_seq = encoder_input_data[seq_index: seq_index + 1]

with open(data_path, 'r', encoding='utf-8') as f:
    test_lines = f.read().split('\n')

test_input_texts=[]
test_target_texts=[]
for line in test_lines[3000: 3100]:
    #print(line)
    test_input_text, test_target_text = line.split(' ')
    #print(test_input_text)
    test_input_texts.append(test_input_text)
    test_target_texts.append(test_target_text+'\n')
    
#print(test_input_texts)   

total_test_input=0
total_accuracy=0

for i in range(50):
    total_test_input=total_test_input+1
    input_seq = get_input_data(test_input_texts[i])
    decoded_sentence = decode_sequence(input_seq)
    if (decoded_sentence==test_target_texts[i]):
        total_accuracy=total_accuracy+1
        #print('hoiche')
        
    print('Input sentence:',   test_input_texts[i])
    print('Decoded sentence:', decoded_sentence)
    #print(decoded_sentence[6])
    print('Target sentence:',  test_target_texts[i])
    #print(test_target_texts[i][6])
  
    
print('total_test_input:', total_test_input)
print('total_accuracy:', total_accuracy)
print('result:', (total_accuracy/total_test_input)*100,'%')

#yhat = model.predict(input_seq, verbose=0)
#print(yhat)

kire


ValueError: Dimension 0 in both shapes must be equal, but are 66 and 63. Shapes are [66,1024] and [63,1024]. for 'Assign_8' (op: 'Assign') with input shapes: [66,1024], [63,1024].

In [21]:
a=2

In [22]:
a=3

In [23]:
print(a)

3
